## Import all require library

In [1]:
import mne
import numpy as np
import pandas as pd
import os
from datetime import datetime, timedelta
import pytz
from utility import *
import json 

# pd.set_option('display.max_rows', 500)

## Look at the structure of the annotation

In [2]:
# Structure of the edf file
edf_filename  = "EEG_Dataset/00357030/00357030.edf"
raw           = mne.io.read_raw_edf(edf_filename, preload = True)
annotation_df = raw.annotations.to_data_frame()
annotation_df

Extracting EDF parameters from c:\Users\xinju\Desktop\Python_Jupyter\Y3_Sem2_Thesis_XAI\EEG_Dataset\00357030\00357030.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 646749  =      0.000 ...  2586.996 secs...


,onset,duration,description
0,2023-06-13 09:46:26.000000,0.000,Impedance
1,2023-06-13 09:46:26.000000,0.000,Detections Inactive
2,2023-06-13 09:46:27.152000,39.728,Calibration
3,2023-06-13 09:47:13.000000,0.000,Montage is now: Com Average (T1-T2)
4,2023-06-13 09:47:20.000000,0.000,ACC N: On
...,...,...,...
76,2023-06-13 10:25:29.608000,0.000,Eyes Closed
77,2023-06-13 10:26:29.883000,0.000,Eyes Open
78,2023-06-13 10:26:47.249600,0.000,Eyes Closed
79,2023-06-13 10:27:27.491600,0.000,Eyes Closed


## Preprocessing the data from edf to csv

### Find duration to skip and spike location

In [3]:
utc              = pytz.UTC
SECONDS_TO_TRASH = 10         # Choosing difference SECONDS_TO_TRASH will have the effect on value inside eeg due to the function (mne.resample) 
dict_spike       = {}
dict_duration    = {}

for patient in os.listdir('EEG_Dataset'):                  # For each of the patient 
    for filename in os.listdir(f"EEG_Dataset/{patient}"):  # For each of the file in the patient folder
        if filename.endswith(".edf"):
            edf_filename = f"EEG_Dataset/{patient}/" + filename
            dict_spike[edf_filename] = []
            print("For the file:",edf_filename)
            
            # Load the edf file and get the annotation of data
            raw               = mne.io.read_raw_edf(edf_filename, preload = True) # Return a raw edf object
            # print(raw.to_data_frame().columns)
            annotation_df     = raw.annotations.to_data_frame()                   # Annotation of the data in dataframe format
            
            # ---------- FIND DURATION TO SKIP ---------- 
            clean_data_before = raw.info["meas_date"]                             # raw.info["meas_date"] = Measurement date

            # Find the calibration duration (if there is any)
            for i, description in enumerate(annotation_df["description"]):  # For each of the descriptions
                if description.lower().__contains__("calibration"):         # If it contain the word 'calibration'
                    print("Description contain 'calibration'",annotation_df.iloc[i,:].to_numpy(), "\n")               

                    # Obtain the start time of the annotation and its duration
                    annotated_start   = annotation_df.iloc[i,0]
                    duration          = timedelta(seconds=annotation_df.iloc[i,1])
                    
                    temp              = annotated_start+duration
                    if temp.replace(tzinfo=utc) > clean_data_before.replace(tzinfo=utc):
                        clean_data_before = temp.replace(tzinfo=utc)
                    
            
            duration_to_skip = (clean_data_before - raw.info["meas_date"]).total_seconds()  # Duration to throw from the measurement date
            duration_to_skip = np.ceil(duration_to_skip) + SECONDS_TO_TRASH
            print("Duration to skip from the measurement date: ", duration_to_skip)


            # ---------- FIND SPIKE LOCATION (if there is any) ----------    
            for i, description in enumerate(annotation_df["description"]):
                if description.lower().__contains__("spike"):                 # If it contain the word 'spike'
                    print("Description contain 'spike'", annotation_df.iloc[i,:].to_numpy())

                    # Obtain the start time of the spike occur
                    annotated_start   = annotation_df.iloc[i,0]
                    dict_spike[edf_filename].append((annotated_start.replace(tzinfo=utc) - raw.info["meas_date"]).total_seconds() - duration_to_skip)

            dict_duration[edf_filename] = duration_to_skip

For the file: EEG_Dataset/00357030/00357030.edf
Extracting EDF parameters from c:\Users\xinju\Desktop\Python_Jupyter\Y3_Sem2_Thesis_XAI\EEG_Dataset\00357030\00357030.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 646749  =      0.000 ...  2586.996 secs...
Description contain 'calibration' [Timestamp('2023-06-13 09:46:27.152000') 39.728 'Calibration'] 

Duration to skip from the measurement date:  51.0
For the file: EEG_Dataset/05648564/05648564.edf
Extracting EDF parameters from c:\Users\xinju\Desktop\Python_Jupyter\Y3_Sem2_Thesis_XAI\EEG_Dataset\05648564\05648564.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 244999  =      0.000 ...   489.998 secs...
Description contain 'calibration' [Timestamp('2023-11-03 12:26:03.100000') 32.84 'Calibration'] 

Description contain 'calibration' [Timestamp('2023-11-03 12:26:15') 0.0 'Montage is now: Calibration'] 

Duration to skip from 

C:\Users\xinju\AppData\Local\Temp\ipykernel_9684\69719431.py:14: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw               = mne.io.read_raw_edf(edf_filename, preload = True) # Return a raw edf object


Duration to skip from the measurement date:  10.0
For the file: EEG_Dataset/36880636/36880636.edf
Extracting EDF parameters from c:\Users\xinju\Desktop\Python_Jupyter\Y3_Sem2_Thesis_XAI\EEG_Dataset\36880636\36880636.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 757999  =      0.000 ...  3031.996 secs...
Description contain 'calibration' [Timestamp('2023-07-04 13:16:29.096000') 40.692 'Calibration'] 

Duration to skip from the measurement date:  52.0
For the file: EEG_Dataset/36946379/369463791.edf
Extracting EDF parameters from c:\Users\xinju\Desktop\Python_Jupyter\Y3_Sem2_Thesis_XAI\EEG_Dataset\36946379\369463791.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 1380863  =      0.000 ...  2696.998 secs...
Duration to skip from the measurement date:  10.0
Description contain 'spike' [Timestamp('2023-03-08 13:32:47.800000') 0.0 'Spike']
Description contain 'spike' [Timestamp('

### Print out duration to skip and spike location for each file

In [4]:
count              = 0
num_spike          = 0
file_contain_spike = []
dict_spike_csv     = {}

for i, (filename, spike) in enumerate(dict_spike.items(), 1):
    if spike != []: 
        count+=1
        num_spike += len(spike)
        file_contain_spike.append(filename)
        
        print(filename, f"\teeg{i}\t", len(spike), spike, "\n")
        dict_spike_csv[f"eeg{i}"] = spike
    else:
        dict_spike_csv[f"eeg{i}"] = []
    
print(f"There is {count} file that contain spike, among them, there is total {num_spike} spikes")


EEG_Dataset/07941048/07941048.edf 	eeg6	 13 [37.6875, 71.59020000000001, 104.85409999999999, 270.9652, 1561.6111, 1705.7361, 2707.2291, 2819.368, 2853.4375, 2884.2222, 3385.1875, 3644.9236, 3736.6458] 

EEG_Dataset/35210490/35210490.edf 	eeg19	 2 [1215.1587, 1494.3819] 

EEG_Dataset/35895458/358954581.edf 	eeg24	 1 [1344.4791] 

EEG_Dataset/36790413/367904131.edf 	eeg31	 31 [57.75, 241.4544, 247.3528, 256.9906, 262.2288, 285.0394, 302.3083, 312.8021, 320.8226, 322.9511, 361.0667, 365.4802, 372.3727, 391.8333, 415.9242, 419.1663, 427.7347, 443.2382, 465.2175, 482.7321, 490.5071, 511.8112, 527.9183, 532.3753, 1133.15, 1219.7916, 1281.175, 1299.5083, 1570.3833, 2570.65, 2641.0666] 

EEG_Dataset/36946379/369463791.edf 	eeg36	 24 [91.8, 191.0125, 193.9187, 256.0312, 289.8157, 329.8375, 333.5562, 358.4125, 387.3504, 516.1907, 566.6699, 632.6907, 717.7937, 790.6597, 816.5562, 922.0, 934.9, 1003.7463, 1060.7437, 1212.8157, 1280.3125, 1495.1351, 1949.0833, 2013.5125] 

EEG_Dataset/37222547/3722

In [5]:
for filename, duration in dict_duration.items():
    print(f"For the file: {filename:<35}, Duration to skip from beginning: {duration} s")

For the file: EEG_Dataset/00357030/00357030.edf  , Duration to skip from beginning: 51.0 s
For the file: EEG_Dataset/05648564/05648564.edf  , Duration to skip from beginning: 43.0 s
For the file: EEG_Dataset/05648564/056485641.edf , Duration to skip from beginning: 10.0 s
For the file: EEG_Dataset/07328940/073289401.edf , Duration to skip from beginning: 10.0 s
For the file: EEG_Dataset/07784910/077849101.edf , Duration to skip from beginning: 10.0 s
For the file: EEG_Dataset/07941048/07941048.edf  , Duration to skip from beginning: 66.0 s
For the file: EEG_Dataset/08436754/084367541.edf , Duration to skip from beginning: 10.0 s
For the file: EEG_Dataset/11670123/116701231.edf , Duration to skip from beginning: 10.0 s
For the file: EEG_Dataset/14073975/140739751.edf , Duration to skip from beginning: 10.0 s
For the file: EEG_Dataset/17035058/170350581.edf , Duration to skip from beginning: 10.0 s
For the file: EEG_Dataset/17545706/175457061.edf , Duration to skip from beginning: 10.0 s

### Save the annotation of spike into json file 

In [6]:
with open(f"{CSV_FILE_DIRC}/annotation.json", "w") as f: 
    json.dump(dict_spike_csv, f)

In [7]:
with open(f"{CSV_FILE_DIRC}/annotation.json", "r") as f:
    dict1 = json.load(f)
    print(dict1)

{'eeg1': [], 'eeg2': [], 'eeg3': [], 'eeg4': [], 'eeg5': [], 'eeg6': [37.6875, 71.59020000000001, 104.85409999999999, 270.9652, 1561.6111, 1705.7361, 2707.2291, 2819.368, 2853.4375, 2884.2222, 3385.1875, 3644.9236, 3736.6458], 'eeg7': [], 'eeg8': [], 'eeg9': [], 'eeg10': [], 'eeg11': [], 'eeg12': [], 'eeg13': [], 'eeg14': [], 'eeg15': [], 'eeg16': [], 'eeg17': [], 'eeg18': [], 'eeg19': [1215.1587, 1494.3819], 'eeg20': [], 'eeg21': [], 'eeg22': [], 'eeg23': [], 'eeg24': [1344.4791], 'eeg25': [], 'eeg26': [], 'eeg27': [], 'eeg28': [], 'eeg29': [], 'eeg30': [], 'eeg31': [57.75, 241.4544, 247.3528, 256.9906, 262.2288, 285.0394, 302.3083, 312.8021, 320.8226, 322.9511, 361.0667, 365.4802, 372.3727, 391.8333, 415.9242, 419.1663, 427.7347, 443.2382, 465.2175, 482.7321, 490.5071, 511.8112, 527.9183, 532.3753, 1133.15, 1219.7916, 1281.175, 1299.5083, 1570.3833, 2570.65, 2641.0666], 'eeg32': [], 'eeg33': [], 'eeg34': [], 'eeg35': [], 'eeg36': [91.8, 191.0125, 193.9187, 256.0312, 289.8157, 329.837

### Preprocess the EEG data
* remain only 19 channels
* apply average montage
* apply band pass filter
* Resample the data from (250, 500, 512) Hz to 128 Hz

In [8]:
%%time
num = 1

for patient in os.listdir('EEG_Dataset'):
    for filename in os.listdir(f"EEG_Dataset/{patient}"):
        if filename.endswith(".edf"):
            if not os.path.exists(f"{CSV_FILE_DIRC}/eeg{num}.csv"): # If end with .edf, and the file haven't been process
                edf_filename = f"EEG_Dataset/{patient}/" + filename
                process_edf(edf_filename, num, 
                            SKIP_FIRST=dict_duration[edf_filename], 
                            SKIP_LAST=SECONDS_TO_TRASH)    
            num += 1

For the file:  EEG_Dataset/00357030/00357030.edf
Extracting EDF parameters from c:\Users\xinju\Desktop\Python_Jupyter\Y3_Sem2_Thesis_XAI\EEG_Dataset\00357030\00357030.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 646749  =      0.000 ...  2586.996 secs...
Before drop some row, shape = (646750, 37)
After  drop some row, shape = (631500, 37)
Setting up band-pass filter from 0.5 - 70 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 70.00 Hz
- Upper transition bandwidth: 17.50 Hz (-6 dB cutoff frequency: 78.75 Hz)
- Filter length: 1651 samples (6.604 s)

Creating RawArray with float64 data, n_channels=19, n_times=631500
    Range : 

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


EEG_Dataset/00357030/00357030.edf does not contain NULL value, the process will continue
The dataframe of EEG_Dataset/00357030/00357030.edf have been saved to EEG_csv/eeg1.csv

For the file:  EEG_Dataset/05648564/05648564.edf
Extracting EDF parameters from c:\Users\xinju\Desktop\Python_Jupyter\Y3_Sem2_Thesis_XAI\EEG_Dataset\05648564\05648564.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 244999  =      0.000 ...   489.998 secs...
Before drop some row, shape = (245000, 37)
After  drop some row, shape = (218500, 37)
Setting up band-pass filter from 0.5 - 70 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 70.00 Hz
- Upper transitio

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


EEG_Dataset/05648564/05648564.edf does not contain NULL value, the process will continue
The dataframe of EEG_Dataset/05648564/05648564.edf have been saved to EEG_csv/eeg2.csv

For the file:  EEG_Dataset/05648564/056485641.edf
Extracting EDF parameters from c:\Users\xinju\Desktop\Python_Jupyter\Y3_Sem2_Thesis_XAI\EEG_Dataset\05648564\056485641.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 451499  =      0.000 ...   902.998 secs...
Before drop some row, shape = (451500, 39)
After  drop some row, shape = (441500, 39)
Setting up band-pass filter from 0.5 - 70 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 70.00 Hz
- Upper transit

...

EEG_Dataset/36979297/369792972.edf does not contain NULL value, the process will continue
The dataframe of EEG_Dataset/36979297/369792972.edf have been saved to EEG_csv/eeg38.csv

For the file:  EEG_Dataset/37222547/372225472.edf
Extracting EDF parameters from c:\Users\xinju\Desktop\Python_Jupyter\Y3_Sem2_Thesis_XAI\EEG_Dataset\37222547\372225472.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 112999  =      0.000 ...   451.996 secs...
Before drop some row, shape = (113000, 37)
After  drop some row, shape = (108000, 37)
Setting up band-pass filter from 0.5 - 70 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 70.00 Hz
- Upper tran

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


EEG_Dataset/37222547/372225472.edf does not contain NULL value, the process will continue
The dataframe of EEG_Dataset/37222547/372225472.edf have been saved to EEG_csv/eeg39.csv

For the file:  EEG_Dataset/37222547/372225473.edf
Extracting EDF parameters from c:\Users\xinju\Desktop\Python_Jupyter\Y3_Sem2_Thesis_XAI\EEG_Dataset\37222547\372225473.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 734749  =      0.000 ...  2938.996 secs...
Before drop some row, shape = (734750, 37)
After  drop some row, shape = (729750, 37)
Setting up band-pass filter from 0.5 - 70 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 70.00 Hz
- Upper tran

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


EEG_Dataset/37222547/372225473.edf does not contain NULL value, the process will continue
The dataframe of EEG_Dataset/37222547/372225473.edf have been saved to EEG_csv/eeg40.csv

CPU times: total: 9min 47s
Wall time: 10min 24s
